In [1]:
# 檔案說明：
# 請將train.csv以及test.csv放在同一個資料夾中，此檔案可以直接 Run All，將輸出一個名為"test_predictions.csv"的檔案，為上傳至競賽網站之檔案
# 但由於當中我們有做train data的shuffle，每次訓練出的model及結果會有些許差異，不完全跟網路上最好結果的檔案一樣

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
train  = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
upload = pd.read_csv("sample_upload.csv")

In [4]:
train['class'] = train['class'].astype('category')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14869 entries, 0 to 14868
Data columns (total 2 columns):
class    14869 non-null category
tweet    14869 non-null object
dtypes: category(1), object(1)
memory usage: 130.9+ KB


In [6]:
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [7]:
train['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in train['tweet']]
test['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',' ',text)) for text in lis]) for lis in test['tweet']]

In [8]:
train.head()

,class,tweet,text_lem
0,1,"[9-1-13] 2:50 pm ""son of a bitch ate my mac n ...",pm son of a bitch ate my mac n ...
1,1,RT @BryceSerna: Don't be a pussy grab the boot...,RT BryceSerna Don t be a pussy grab the boot...
2,2,RT @ClicquotSuave: bunch of rappers boutta flo...,RT ClicquotSuave bunch of rappers boutta flo...
3,2,@michigannews13 wow. Thats great language comi...,michigannews wow Thats great language comi...
4,1,"and this is why I'm single, I don't fuck with ...",and this is why I m single I don t fuck with ...


In [9]:
test.head()

,id,tweet,text_lem
0,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,RT UrKindOfBrand Dawg RT sbaby...
1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,RT C G Anderson viva based she lo...
2,2,"!!!!!!""@__BrighterDays: I can not just sit up ...",BrighterDays I can not just sit up ...
3,3,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,selfiequeenbri cause I m tired of...
4,4,""" @rhythmixx_ :hobbies include: fighting Maria...",rhythmixx hobbies include fighting Maria...


In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
vect = TfidfVectorizer(ngram_range = (1,4)).fit(train['text_lem'])
vect_transformed_train = vect.transform(train['text_lem'])
vect_transformed_test = vect.transform(test['text_lem'])

In [12]:
# Ensemble Learning
train_class0=train[train['class']==0]
train_class1=train[train['class']==1]
train_class2=train[train['class']==2]

from sklearn.utils import shuffle
train_class1 = shuffle(train_class1)

train_class1_1=train_class1.iloc[0:2800,:]
train_class1_2=train_class1.iloc[2801:5600,:]
train_class1_3=train_class1.iloc[5601:8400,:]
train_class1_4=train_class1.iloc[8401:11491,:]

train1=pd.concat([train_class0,train_class2,train_class1_1],axis=0)
train2=pd.concat([train_class0,train_class2,train_class1_2],axis=0)
train3=pd.concat([train_class0,train_class2,train_class1_3],axis=0)
train4=pd.concat([train_class0,train_class2,train_class1_4],axis=0)

vect_transformed_train1 = vect.transform(train1['text_lem'])
FinalModel = LogisticRegression(C=250).fit(vect_transformed_train1,train1['class'])
predictions1 = FinalModel.predict_proba(vect_transformed_test)
predictions1_train = FinalModel.predict_proba(vect_transformed_train)
vect_transformed_train2 = vect.transform(train2['text_lem'])
FinalModel = LogisticRegression(C=250).fit(vect_transformed_train2,train2['class'])
predictions2 = FinalModel.predict_proba(vect_transformed_test)
predictions2_train = FinalModel.predict_proba(vect_transformed_train)
vect_transformed_train3 = vect.transform(train3['text_lem'])
FinalModel = LogisticRegression(C=250).fit(vect_transformed_train3,train3['class'])
predictions3 = FinalModel.predict_proba(vect_transformed_test)
predictions3_train = FinalModel.predict_proba(vect_transformed_train)
vect_transformed_train4 = vect.transform(train4['text_lem'])
FinalModel = LogisticRegression(C=250).fit(vect_transformed_train4,train4['class'])
predictions4 = FinalModel.predict_proba(vect_transformed_test)
predictions4_train = FinalModel.predict_proba(vect_transformed_train)

predictions_analysis_train=np.concatenate((predictions1_train,predictions2_train,predictions3_train,predictions4_train),axis=1)


predictions_analysis=np.concatenate((predictions1,predictions2,predictions3,predictions4),axis=1)

from sklearn.model_selection import GridSearchCV
parameters = {
    'penalty': ('l1', 'l2'),
    'C': (0.1, 1, 10)
}
LR = LogisticRegression()
FinalModel=GridSearchCV(LR, parameters, cv=5)
FinalModel.fit(predictions_analysis_train,train['class'])
print(FinalModel.best_score_)
print(FinalModel.best_params_)

predictions = FinalModel.predict(predictions_analysis)

0.999663729907862
{'C': 0.1, 'penalty': 'l1'}


In [13]:
submission = pd.DataFrame({'id':test['id'],'class':predictions})

In [14]:
file_name = 'test_predictions.csv'
submission.to_csv(file_name,index=False)